In [1]:
%pip install catboost scikit-learn pandas numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# IMPORTANDO BIBLIOTECAS

# MANEJAR DADOS
import pandas as pd
import numpy as np

# 2. PLOTAR DADOS
import matplotlib.pyplot as plt


# 3. PREPROCESSAMENTO DOS DADOS
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


# 5. Bibliotecas para separamento de cojuntos de dados
from sklearn.model_selection import train_test_split,cross_val_predict,GridSearchCV, cross_val_score,KFold,StratifiedKFold
from imblearn.over_sampling import SMOTE

# 6. Algoritmos de classificação
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier,RandomForestRegressor, VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

#importing pipeline
from sklearn.pipeline import Pipeline

import pickle

# 7. Métricas
from sklearn.metrics import accuracy_score, classification_report,precision_score, recall_score, f1_score , mean_absolute_error, mean_squared_error

# 8. Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# EDA

In [3]:
df = pd.read_csv("./Heart_disease_cleveland_new.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,0,145,233,1,2,150,0,2.3,2,0,2,0
1,67,1,3,160,286,0,2,108,1,1.5,1,3,1,1
2,67,1,3,120,229,0,2,129,1,2.6,1,2,3,1
3,37,1,2,130,250,0,0,187,0,3.5,2,0,1,0
4,41,0,1,130,204,0,2,172,0,1.4,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
X= df.drop(['target'], axis=1)
y = df['target']

In [6]:
categorical_columns = ['thal', 'ca', 'slope', 'exang', 'restecg', 'cp', 'sex']

In [7]:
X_encoded = X.copy()
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    X_encoded[col] = label_encoders[col].fit_transform(X[col])



In [8]:
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_encoded, y,)

# Ensemble Learning

In [11]:

params_rf = {"criterion": "gini", "max_depth": 8, "max_features": "auto", "n_estimators": 200}
params_rf = {}
params_catboost = {'n_estimators': 300, 'learning_rate': 0.001, 'random_state': 42, 'verbose': 0, 'depth': 4, 'loss_function': 'Logloss', 'class_weights': [1, 1.15]}
params_ab = {'n_estimators': 250,  'learning_rate': 0.6, 'random_state':42}
params_nb = {'var_smoothing': 1e-09}
params_xgb={'reg_lambda': 2, 'reg_alpha': 0.1, 'n_estimators': 300, 'n_estimator': 200, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0, 'eval_metric': 'logloss', 'colsample_bytree': 0.6}
logistic_rg_params = {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 10}
params_gb = {'n_estimators': 200, 'min_samples_split': 5, 'max_depth': 5, 'learning_rate': 0.2}

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
rf_model = RandomForestClassifier(**params_rf)
catboost_model = CatBoostClassifier(**params_catboost)
xgb_model = XGBClassifier(**params_xgb)
nb_model = GaussianNB(**params_nb)
ab_model = AdaBoostClassifier(**params_ab)
lg_model = LogisticRegression(**logistic_rg_params)
gb_model = GradientBoostingClassifier(**params_gb)

# ensemble_weights =  [1, 2, 1, 1, 2, 1]
# ensemble_weights = [1, 4, 1, 1, 2, 1]
ensemble_weights = [1, 7, 1, 1, 2, 1]
TEST_SIZE = 0.1
ensemble = VotingClassifier(estimators=[
    ('lg', lg_model),
    ('xgb', xgb_model),
    ('rf', rf_model),
    ('catboost', catboost_model),
    ('ab', ab_model),
    ('nb', nb_model)
    ], voting='hard', weights=ensemble_weights)

X_train, X_test, y_train, y_test = train_test_split(X_encoded.values, y, test_size=TEST_SIZE, random_state=42)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
ensemble.fit(X_train_resampled, y_train_resampled)

y_pred = ensemble.predict(X_test)
# Avaliar as métricas
accuracy = (accuracy_score(y_test, y_pred))
print(accuracy)

0.967741935483871


In [ ]:

with open('trained_model-0.1.0.pkl', 'wb') as f:
    pickle.dump(ensemble, f)
